### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [1]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def extract_words(sent):
    #print("line tokenizing : \n{0}".format(sent))
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [4]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [5]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.7402668]], dtype=float32)

In [6]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("file being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [7]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

file being read : ../indata/eminem-not-affraid.csv
file being read : ../indata/got8.csv
file being read : ../indata/house-of-cards.csv
file being read : ../indata/samsung-vr.csv
file being read : ../indata/tonight-show.csv
file being read : ../indata/tswift.csv
file being read : ../indata/warriors.csv


In [8]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[quien, escucha, en]","[-0.017592942, 0.1241421, -0.08706584, 0.05988..."
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[i, m, not, afraid, ok, oscar, the, grouch, go...","[0.41558224, -0.044828884, 0.50611204, -0.2152..."
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,[eminemcia],"[0.004891281, -0.0026302517, 0.007052113, -0.0..."
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[my, favorite, rapper, of, all, time]","[0.02787096, 0.26540336, 0.07880832, -0.283423..."
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[he, was, probably, shitting, his, pants, on, ...","[-0.18669322, 0.31303778, 0.08214769, -0.13526..."


In [9]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['eminem-not-affraid.csv' 'got8.csv' 'house-of-cards.csv' 'samsung-vr.csv'
 'tonight-show.csv' 'tswift.csv' 'warriors.csv']
No of Type : ['Comment' 'Reply']


In [10]:
trained_clf = 'doc2vec_rf.model'
loaded_clf = joblib.load(trained_clf)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
comments['SENTIMENT_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [12]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS,SENTIMENT_RF
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[quien, escucha, en]","[-0.017592942, 0.1241421, -0.08706584, 0.05988...",1
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[i, m, not, afraid, ok, oscar, the, grouch, go...","[0.41558224, -0.044828884, 0.50611204, -0.2152...",0
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,[eminemcia],"[0.004891281, -0.0026302517, 0.007052113, -0.0...",0
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[my, favorite, rapper, of, all, time]","[0.02787096, 0.26540336, 0.07880832, -0.283423...",1
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[he, was, probably, shitting, his, pants, on, ...","[-0.18669322, 0.31303778, 0.08214769, -0.13526...",0


In [ ]:
trained_clf = 'doc2vec_kn.model'
loaded_clf = joblib.load(trained_clf)

In [ ]:
comments['SENTIMENT_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [ ]:
comments.head(5)

In [ ]:
out_df = comments[['Comment','CreateTimeStamp','Type', 'videoID','SENTIMENT_RF','SENTIMENT_KN']]

In [ ]:
out_df.head(5)

In [ ]:
out_file_name = '../outdata/video_comments_analysis_sentiments.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [ ]:
print('****End of script****')